In [31]:
import pandas as pd
import numpy as np
import re
from sklearn.manifold import TSNE
from functools import reduce
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

random_state = 1

theme = ["#d2fbd4","#a5dbc2","#7bbcb0","#559c9e","#3a7c89","#235d72","#123f5a"]

In [32]:
cosmetic = pd.read_csv(r"C:\Users\flore\Downloads\skincare\skincare_prepared.csv", sep=",")
cosmetic

,product_name,brand,product_url,product_type,ingredients,price,contents
0,Acorelle Pure Harvest Body Perfume - 100ml,Acorelle,https://www.lookfantastic.com/acorelle-pure-ha...,Mist,"Alcohol, Aqua, Glycerin, Fragrance, Limonene, ...",10.0,100ml
1,Aesop Parsley Seed Anti-Oxidant Eye Cream 10ml,Aesop,https://www.lookfantastic.com/aesop-parsley-se...,Eye Care,"Aloe Barbadensis Leaf Juice, Water, PEG-60 Alm...",63.0,10ml
2,Aesop Parsley Seed Anti-Oxidant Eye Serum 15ml,Aesop,https://www.lookfantastic.com/aesop-parsley-se...,Eye Care,"Aloe Babedensis Lead Juice, Water (Aqua), PEG ...",57.0,15ml
3,Aesop Amazing Face Cleanser 200ml,Aesop,https://www.lookfantastic.com/aesop-amazing-fa...,Cleanser,"Water, Cocamidopropyl Belaine, Sea Salt, Glyce...",35.0,200ml
4,Aesop Animal Body Wash 500ml,Aesop,https://www.lookfantastic.com/aesop-animal-bod...,Body Wash,"Water (Aqua), Sodium Laureth Sulfate, Propylen...",27.0,500ml
...,...,...,...,...,...,...,...
1130,Zelens Triple Action Advanced Eye Cream,Zelens,https://www.lookfantastic.com/zelens-triple-ac...,Eye Care,"Aqua (Water), Dimethicone, Cyclopentasiloxane,...",80.0,15ml
1131,Zelens Youth Concentrate Supreme Age-Defying S...,Zelens,https://www.lookfantastic.com/zelens-youth-con...,Serum,"Water (Aqua), Caprylic/Capric Triglyceride, Hy...",160.0,30ml
1132,Zelens Z Hyaluron Hyaluronic Acid Complex Seru...,Zelens,https://www.lookfantastic.com/zelens-z-hyaluro...,Serum,"Aqua (Water), Glycerin, Propanediol, Hydrolyze...",55.0,30ml
1133,Zelens PROVITAMIN D Fortifying Facial Mist 50ml,Zelens,https://www.lookfantastic.com/zelens-provitami...,Mist,"Aqua (Water), Propanediol, Glycerin, Polyglyce...",55.0,50ml


In [33]:
cosmetic['contents'].isna().sum()

0

In [34]:
cosmetic['contents'] = cosmetic['contents'].str.replace('ml', '')
cosmetic['contents'] = cosmetic['contents'].str.replace('g', '')
cosmetic

,product_name,brand,product_url,product_type,ingredients,price,contents
0,Acorelle Pure Harvest Body Perfume - 100ml,Acorelle,https://www.lookfantastic.com/acorelle-pure-ha...,Mist,"Alcohol, Aqua, Glycerin, Fragrance, Limonene, ...",10.0,100
1,Aesop Parsley Seed Anti-Oxidant Eye Cream 10ml,Aesop,https://www.lookfantastic.com/aesop-parsley-se...,Eye Care,"Aloe Barbadensis Leaf Juice, Water, PEG-60 Alm...",63.0,10
2,Aesop Parsley Seed Anti-Oxidant Eye Serum 15ml,Aesop,https://www.lookfantastic.com/aesop-parsley-se...,Eye Care,"Aloe Babedensis Lead Juice, Water (Aqua), PEG ...",57.0,15
3,Aesop Amazing Face Cleanser 200ml,Aesop,https://www.lookfantastic.com/aesop-amazing-fa...,Cleanser,"Water, Cocamidopropyl Belaine, Sea Salt, Glyce...",35.0,200
4,Aesop Animal Body Wash 500ml,Aesop,https://www.lookfantastic.com/aesop-animal-bod...,Body Wash,"Water (Aqua), Sodium Laureth Sulfate, Propylen...",27.0,500
...,...,...,...,...,...,...,...
1130,Zelens Triple Action Advanced Eye Cream,Zelens,https://www.lookfantastic.com/zelens-triple-ac...,Eye Care,"Aqua (Water), Dimethicone, Cyclopentasiloxane,...",80.0,15
1131,Zelens Youth Concentrate Supreme Age-Defying S...,Zelens,https://www.lookfantastic.com/zelens-youth-con...,Serum,"Water (Aqua), Caprylic/Capric Triglyceride, Hy...",160.0,30
1132,Zelens Z Hyaluron Hyaluronic Acid Complex Seru...,Zelens,https://www.lookfantastic.com/zelens-z-hyaluro...,Serum,"Aqua (Water), Glycerin, Propanediol, Hydrolyze...",55.0,30
1133,Zelens PROVITAMIN D Fortifying Facial Mist 50ml,Zelens,https://www.lookfantastic.com/zelens-provitami...,Mist,"Aqua (Water), Propanediol, Glycerin, Polyglyce...",55.0,50


In [35]:
cosmetic['contents'] = cosmetic['contents'].astype(float)
cosmetic['price'] = cosmetic['price'].astype(float)
cosmetic

,product_name,brand,product_url,product_type,ingredients,price,contents
0,Acorelle Pure Harvest Body Perfume - 100ml,Acorelle,https://www.lookfantastic.com/acorelle-pure-ha...,Mist,"Alcohol, Aqua, Glycerin, Fragrance, Limonene, ...",10.0,100.0
1,Aesop Parsley Seed Anti-Oxidant Eye Cream 10ml,Aesop,https://www.lookfantastic.com/aesop-parsley-se...,Eye Care,"Aloe Barbadensis Leaf Juice, Water, PEG-60 Alm...",63.0,10.0
2,Aesop Parsley Seed Anti-Oxidant Eye Serum 15ml,Aesop,https://www.lookfantastic.com/aesop-parsley-se...,Eye Care,"Aloe Babedensis Lead Juice, Water (Aqua), PEG ...",57.0,15.0
3,Aesop Amazing Face Cleanser 200ml,Aesop,https://www.lookfantastic.com/aesop-amazing-fa...,Cleanser,"Water, Cocamidopropyl Belaine, Sea Salt, Glyce...",35.0,200.0
4,Aesop Animal Body Wash 500ml,Aesop,https://www.lookfantastic.com/aesop-animal-bod...,Body Wash,"Water (Aqua), Sodium Laureth Sulfate, Propylen...",27.0,500.0
...,...,...,...,...,...,...,...
1130,Zelens Triple Action Advanced Eye Cream,Zelens,https://www.lookfantastic.com/zelens-triple-ac...,Eye Care,"Aqua (Water), Dimethicone, Cyclopentasiloxane,...",80.0,15.0
1131,Zelens Youth Concentrate Supreme Age-Defying S...,Zelens,https://www.lookfantastic.com/zelens-youth-con...,Serum,"Water (Aqua), Caprylic/Capric Triglyceride, Hy...",160.0,30.0
1132,Zelens Z Hyaluron Hyaluronic Acid Complex Seru...,Zelens,https://www.lookfantastic.com/zelens-z-hyaluro...,Serum,"Aqua (Water), Glycerin, Propanediol, Hydrolyze...",55.0,30.0
1133,Zelens PROVITAMIN D Fortifying Facial Mist 50ml,Zelens,https://www.lookfantastic.com/zelens-provitami...,Mist,"Aqua (Water), Propanediol, Glycerin, Polyglyce...",55.0,50.0


In [36]:
cosmetic['product_type'].unique()

array(['Mist', 'Eye Care', 'Cleanser', 'Body Wash', 'Mask', 'Bath Salts',
       'Peel', 'Toner', 'Balm', 'Moisturiser', 'Bath Oil', 'Oil', 'Serum',
       'Exfoliator'], dtype=object)

In [37]:
print(cosmetic['price'].describe())

count    1135.000000
mean       23.963419
std        21.713618
min         1.950000
25%         9.950000
50%        18.900000
75%        31.250000
max       230.000000
Name: price, dtype: float64


In [38]:
cosmetic['price'] = cosmetic["price"].apply(lambda x: 1 if x > 18.90 else 0)
cosmetic['price'].value_counts()

0    568
1    567
Name: price, dtype: int64

In [39]:
y = np.array(cosmetic['price'])
y.shape
y

array([0, 1, 1, ..., 1, 1, 1], dtype=int64)

In [40]:
def tokenizer(x) -> list: 
    x = x.replace('(', '')
    x = x.replace(')', '')
    x = x.replace("\xa0", " ")
    x = x.replace(".", ",")
    x = x.replace(" & ", ", ")
    x = re.split(', ', x)
    
    return x

In [41]:
tokenizer("(Hello.\xa0 & Humans!")

['Hello', '', 'Humans!']

In [42]:
ohe = OneHotEncoder()
vect = CountVectorizer(tokenizer=lambda x: tokenizer(x))
scaler = StandardScaler()

In [43]:
X = cosmetic[['brand', 'product_type']]
X

Xtraining, Xtesting, ytraining, ytesting= train_test_split(X, y, 
                                                    test_size = 0.4, random_state=random_state, stratify=y)

### LOGISTIC REGRESSION without ingredients and content

In [44]:
columnT = make_column_transformer(
    (ohe, ["product_type"]),
    (vect, "brand"), 
    remainder="passthrough")

In [45]:
log = LogisticRegression(random_state=random_state, max_iter = 10000)
logPipe = make_pipeline(columnT, log)

In [46]:
logPipe.fit(Xtraining ,ytraining)
logPredictions = logPipe.predict(Xtesting)

In [47]:
accuracy_score(ytesting, logPredictions)

0.8303964757709251

In [48]:
solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
tols = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
Cs = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
param_grid = {'logisticregression__C': Cs, 'logisticregression__solver':solvers, 'logisticregression__tol':tols}
logPipe = make_pipeline(columnT, log)
lrgs = RandomizedSearchCV(estimator=logPipe, param_distributions=param_grid, random_state=random_state, scoring='accuracy', cv=10, refit=True)
lrgs = lrgs.fit(Xtraining, ytraining)
lrgs.best_params_

{'logisticregression__tol': 1e-05,
 'logisticregression__solver': 'saga',
 'logisticregression__C': 10}

In [49]:
bestregression = lrgs.best_estimator_ 
bestregression.score(Xtesting,ytesting)

0.8414096916299559

In [50]:
scores = cross_val_score(estimator=bestregression, X=Xtraining, y=ytraining, cv=10, n_jobs=-1)
print(scores)
print('Accuracy = {:.8f} +/- {:.8f}'.format(np.mean(scores), np.std(scores)))

[0.86956522 0.80882353 0.86764706 0.85294118 0.83823529 0.79411765
 0.85294118 0.79411765 0.76470588 0.88235294]
Accuracy = 0.83254476 +/- 0.03750473


### DECISION TREE with ingredients and content value

In [51]:
from sklearn.tree import DecisionTreeClassifier

decisionTree = DecisionTreeClassifier(random_state=random_state, criterion='entropy')

columnT = make_column_transformer(
    (ohe, ["product_type"]),
    (vect, "brand"),
    remainder="passthrough")

treepipe = make_pipeline(columnT, decisionTree)
treepipe.fit(Xtraining, ytraining)
treepredictions = treepipe.predict(Xtesting)
accuracy_score(ytesting, treepredictions)

0.8215859030837004

In [52]:
criterions = ['gini', 'entropy']
max_depths = [None, 10, 100, 1000, 10000]
param_grid = {'decisiontreeclassifier__min_samples_leaf': np.arange(1,50), 
              'decisiontreeclassifier__max_depth': max_depths, 
              'decisiontreeclassifier__criterion':criterions}

dtgs = RandomizedSearchCV(estimator=treepipe, param_distributions=param_grid, random_state=random_state, scoring='accuracy', cv=10, refit=True)
dtgs = dtgs.fit(Xtraining, ytraining)
print(dtgs.best_params_)
besttree = dtgs.best_estimator_ 
print(besttree.score(Xtesting,ytesting))

{'decisiontreeclassifier__min_samples_leaf': 2, 'decisiontreeclassifier__max_depth': 10000, 'decisiontreeclassifier__criterion': 'gini'}
0.7995594713656388


In [53]:
scores = cross_val_score(estimator=besttree, X=Xtraining, y=ytraining, cv=10, n_jobs=-1)
print(scores)
print('Accuracy = {:.8f} +/- {:.8f}'.format(np.mean(scores), np.std(scores)))

[0.76811594 0.80882353 0.76470588 0.80882353 0.79411765 0.66176471
 0.75       0.72058824 0.67647059 0.75      ]
Accuracy = 0.75034101 +/- 0.04844292


### RANDOM FOREST without Ingredient and content value

In [54]:
from sklearn.ensemble import RandomForestClassifier
randomForest = RandomForestClassifier(n_estimators = 1000, random_state = random_state)

randompipe = make_pipeline(columnT, randomForest)

In [55]:
randompipe.fit(Xtraining, ytraining)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['product_type']),
                                                 ('countvectorizer',
                                                  CountVectorizer(tokenizer=<function <lambda> at 0x000001F69FE62E50>),
                                                  'brand')])),
                ('randomforestclassifier',
                 RandomForestClassifier(n_estimators=1000, random_state=1))])

In [56]:
randompred = randompipe.predict(Xtesting)

In [57]:
accuracy_score(ytesting, randompred)

0.8348017621145375

In [58]:
criterions = ['gini', 'entropy']
max_depths = [None, 10, 100, 1000, 10000]
param_grid = {'randomforestclassifier__min_samples_leaf': np.arange(1,50), 
              'randomforestclassifier__max_depth': max_depths, 
              'randomforestclassifier__criterion':criterions}
randomForest = RandomForestClassifier(n_estimators = 1000, random_state = random_state)
randompipe = make_pipeline(columnT, randomForest)

rfgs = RandomizedSearchCV(estimator=randompipe, param_distributions=param_grid, random_state=random_state, scoring='accuracy', cv=10, refit=True)
rfgs = rfgs.fit(Xtraining,ytraining)
print(rfgs.best_params_)
bestforest = rfgs.best_estimator_ 
print(bestforest.score(Xtesting, ytesting))

{'randomforestclassifier__min_samples_leaf': 2, 'randomforestclassifier__max_depth': 10000, 'randomforestclassifier__criterion': 'gini'}
0.7775330396475771


In [59]:
scores = cross_val_score(estimator=bestforest, X=Xtraining, y=ytraining, cv=10, n_jobs=-1)
print(scores)
print('Accuracy = {:.8f} +/- {:.8f}'.format(np.mean(scores), np.std(scores)))

[0.82608696 0.77941176 0.83823529 0.73529412 0.76470588 0.70588235
 0.82352941 0.77941176 0.73529412 0.80882353]
Accuracy = 0.77966752 +/- 0.04238528


### SUPPORT VECTOR MACHINE without ingredients or content value

In [60]:
from sklearn.svm import SVC
svm = SVC(kernel='linear', random_state = random_state)

svmPipe = make_pipeline(columnT, svm)
svmPipe.fit(Xtraining, ytraining)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['product_type']),
                                                 ('countvectorizer',
                                                  CountVectorizer(tokenizer=<function <lambda> at 0x000001F69FE62E50>),
                                                  'brand')])),
                ('svc', SVC(kernel='linear', random_state=1))])

In [61]:
svmPredictions = svmPipe.predict(Xtesting)
accuracy_score(ytesting, svmPredictions)

0.8502202643171806

In [62]:
svm = SVC(kernel='linear', random_state = random_state) 
columnT = make_column_transformer(
    (ohe, ["product_type"]),
    (vect, "brand"), 
    remainder="passthrough")

svmPipe = make_pipeline(columnT, svm)

param_grid = [{'svc__C': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000],
               'svc__kernel': ['linear']},
              {'svc__C': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000],
               'svc__gamma': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000],
               'svc__kernel': ['rbf']}]

svmgs = RandomizedSearchCV(estimator=svmPipe, random_state = random_state, param_distributions=param_grid, scoring='accuracy', cv=10, refit=True)
svmgs = svmgs.fit(Xtraining, ytraining)
print(svmgs.best_params_)
bestsvm = svmgs.best_estimator_ 
print(bestsvm.score(Xtesting, ytesting))

{'svc__kernel': 'rbf', 'svc__gamma': 0.0001, 'svc__C': 10000}
0.8458149779735683


In [63]:
scores = cross_val_score(estimator=bestsvm, X=Xtraining, y=ytraining, cv=10, n_jobs=-1)
print(scores)
print('Accuracy = {:.8f} +/- {:.8f}'.format(np.mean(scores), np.std(scores)))

[0.86956522 0.76470588 0.85294118 0.85294118 0.83823529 0.79411765
 0.86764706 0.77941176 0.70588235 0.82352941]
Accuracy = 0.81489770 +/- 0.05037799


### K- NEAREST NEIGHBOUR without ingredients and content value

In [64]:
from sklearn.neighbors import KNeighborsClassifier as KNN
knn = KNN()

In [65]:
columnT = make_column_transformer(
    (ohe, ["product_type"]),
    (vect, "brand"), 
    remainder="passthrough")

knnPipe = make_pipeline(columnT, knn)
knnPipe.fit(Xtraining, ytraining)
knnPredictions = knnPipe.predict(Xtesting)
accuracy_score(ytesting, knnPredictions)

0.737885462555066

In [66]:
knnPipe = make_pipeline(columnT, knn)
param_grid = {'kneighborsclassifier__n_neighbors': np.arange(1,50), 
              'kneighborsclassifier__weights':['uniform', 'distance']}

knngs = RandomizedSearchCV(estimator=knnPipe, param_distributions=param_grid, random_state=random_state, scoring='accuracy', cv=10, refit=True)
knngs = knngs.fit(Xtraining, ytraining)
print(knngs.best_params_)
bestknn = knngs.best_estimator_ 
print(bestknn.score(Xtesting,ytesting))

{'kneighborsclassifier__weights': 'distance', 'kneighborsclassifier__n_neighbors': 39}
0.8370044052863436


In [67]:
scores = cross_val_score(estimator=bestknn, X=Xtraining, y=ytraining, cv=10, n_jobs=-1)
print(scores)
print('Accuracy = {:.8f} +/- {:.8f}'.format(np.mean(scores), np.std(scores)))

[0.82608696 0.83823529 0.85294118 0.82352941 0.83823529 0.66176471
 0.73529412 0.76470588 0.76470588 0.75      ]
Accuracy = 0.78554987 +/- 0.05758467
